In [72]:
import pickle
import numpy as np 

In [73]:
with open ("train_qa.txt","rb") as fp:
    train_data = pickle.load(fp)

In [ ]:
train_data

In [75]:
with open ("test_qa.txt","rb") as fp:
    test_data = pickle.load(fp)

In [ ]:
test_data

In [ ]:
type(test_data)

In [ ]:
type(train_data)

In [ ]:
len(test_data)

In [ ]:
len(train_data)

In [ ]:
train_data[0]

In [ ]:
" ".join(train_data[0][0])

In [83]:
#set up vocabulary
vocab = set()


In [84]:
all_data = test_data + train_data

In [ ]:
all_data

In [86]:
for story , question , answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [87]:
vocab.add('yes')
vocab.add('no')

In [ ]:
vocab

In [ ]:
len(vocab)

In [90]:
vocab_len = len(vocab)+1

In [ ]:
max_story_len = max([len(data[0]) for data in all_data]) 
max_story_len

In [ ]:
max_ques_len = max([len(data[1]) for data in all_data]) 
max_ques_len

In [93]:
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


In [94]:
tokenizer = Tokenizer(filters= [])

In [95]:
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [97]:
train_story_text = []
train_question_text = []
#train_answers = []


for story , question , answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)


In [98]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [ ]:
len(train_story_text)

In [ ]:
len(train_story_seq)

In [ ]:
train_story_seq

In [102]:
def vectorize_stories(data, word_index = tokenizer.word_index , max_story_len = max_story_len , max_ques_len = max_ques_len):

  X =[]
  Xq =[]
  Y =[]

  for story, question, answer in data:
     X.append([word_index[word.lower()] for word in story])
     Xq.append([word_index[word.lower()] for word in question])
     y = np.zeros(len(word_index) + 1)
     y[word_index[answer]] = 1
     Y.append(y)
  
  return(pad_sequences(X, maxlen = max_story_len),
         pad_sequences(Xq, maxlen = max_ques_len),
         np.array(Y))
 


In [103]:
inputs_train, queries_train , answers_train = vectorize_stories(train_data)

In [104]:
inputs_test, queries_test , answer_test = vectorize_stories(test_data)

In [ ]:
inputs_train

In [ ]:
queries_test

In [107]:
from keras.models import Sequential , Model
from keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add , dot, concatenate, LSTM

In [108]:
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))

In [109]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [110]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_ques_len))
input_encoder_m.add(Dropout(0.3))

In [111]:
#question encoder
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len, output_dim=64, input_shape=(max_ques_len,)))
question_encoder.add(Dropout(0.3))

In [112]:
input_encoded_m = input_encoder_m(input_sequence)

input_encoded_c = input_encoder_c(input_sequence)

question_encoded= question_encoder(question)


In [113]:
match = dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax')(match)

In [114]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)

In [115]:
answer = concatenate([response , question_encoded])

In [ ]:
answer

In [117]:
answer = LSTM(32)(answer)

In [118]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)
answer = Activation('softmax')(answer)

In [119]:
model = Model([input_sequence,question],answer)
model.compile(optimizer= "rmsprop" , loss = 'categorical_crossentropy',metrics= ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model_out = model.fit([inputs_train , queries_train],answers_train ,
           batch_size= 30 , epochs =25 ,
           validation_data=([inputs_test , queries_test] , answer_test))

In [ ]:
import matplotlib.pyplot as plt
print(model_out.history.keys())
plt.plot(model_out.history['accuracy'])
plt.plot(model_out.history['val_accuracy'])


In [124]:
model.save("chat_bot.keras")

In [ ]:
predict_results= model.predict([inputs_test , queries_test])

In [ ]:
predict_results

In [ ]:
val_max = np.argmax(predict_results[37])
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print('Predicted answer is ',k)
print('Probability of cirtainity ', predict_results[37][val_max])